# 구별 데이터 도출
- 연령대별 생활인구 평균 > 구별로 평균
    - 월별 데이터 합산 >> 구별 평균
    
- 가구원별 가구수 > 구별 평균
- 연령대별 거주인구수 평균 > 구별 평균
- 식품위생업소 > 구별 집계

In [36]:
import pandas as pd

PATH = './전처리데이터/'
dfs = pd.read_csv(PATH + '서울_행정동_법정동_맵핑데이터.csv')
df1 = pd.read_csv(PATH + '2021_생활인구_구별_집계.csv')
df2 = pd.read_csv(PATH + '행정동_가구인원별_통계.csv')
df3 = pd.read_csv(PATH + '2021_구별_거주인구수.csv')
df4 = pd.read_csv(PATH + '동별_식품위생업수_최종.csv')

trash = pd.read_excel(PATH + '2021_쓰레기_배출량.xlsx')
trash = trash.drop('연도', axis = 1)
trash.rename(columns = {'지역구':'자치구'}, inplace = True)

trash.head()

,자치구,PET 총량,PET 량(공공처리),PET 량(위탁),CAN 총량,CAN 량(공공처리),CAN 량(위탁)
0,종로구,738.6,219.3,519.3,443.5,138.9,304.6
1,중구,712.8,565.9,146.9,484.4,373.3,111.1
2,용산구,973.0,764.8,208.2,469.9,301.8,168.1
3,성동구,1671.3,1149.5,521.8,931.7,454.6,477.1
4,광진구,1315.5,0.0,1315.5,552.8,0.0,552.8


#### 자치구 기준 평균 집계 / 불필요 컬럼 삭제

In [37]:
df2_1 = df2.groupby('자치구').mean().reset_index()
df2_1.drop('행정동_코드', axis = 1, inplace = True)
df2_fin = df2_1.copy()

In [38]:
df1 = df1.drop('년도', axis = 1)

In [39]:
df4['행정동_코드'] = df4['행정동_코드'].astype(str)

In [40]:
# 행정동 코드 수정
dfs['ADSTRD_CD'] = dfs['ADSTRD_CD'].astype(str)
dfs['ADSTRD_CD'] = dfs['ADSTRD_CD'].str[:-2]
dfs.head()

,CTPV_NM,CTGG_NM,LGDNG_NM,LGDNG_CD,ADSTRD_NM,ADSTRD_CD
0,서울특별시,종로구,종로구,1111000000,종로구,11110000
1,서울특별시,종로구,청운동,1111010100,청운효자동,11110515
2,서울특별시,종로구,신교동,1111010200,청운효자동,11110515
3,서울특별시,종로구,궁정동,1111010300,청운효자동,11110515
4,서울특별시,종로구,효자동,1111010400,청운효자동,11110515


In [41]:
# 데이터 Merge
df4_gu = pd.merge(df4, dfs, left_on='행정동_코드', right_on='ADSTRD_CD')
df4_gu.head()

,행정동_코드,행정동,유흥주점영업,단란주점,제과점영업,집단급식소,일반음식점,휴게음식점,CTPV_NM,CTGG_NM,LGDNG_NM,LGDNG_CD,ADSTRD_NM,ADSTRD_CD
0,11110515,청운효자동,0.0,0.0,75.0,282.0,3057.0,959.0,서울특별시,종로구,청운동,1111010100,청운효자동,11110515
1,11110515,청운효자동,0.0,0.0,75.0,282.0,3057.0,959.0,서울특별시,종로구,신교동,1111010200,청운효자동,11110515
2,11110515,청운효자동,0.0,0.0,75.0,282.0,3057.0,959.0,서울특별시,종로구,궁정동,1111010300,청운효자동,11110515
3,11110515,청운효자동,0.0,0.0,75.0,282.0,3057.0,959.0,서울특별시,종로구,효자동,1111010400,청운효자동,11110515
4,11110515,청운효자동,0.0,0.0,75.0,282.0,3057.0,959.0,서울특별시,종로구,창성동,1111010500,청운효자동,11110515


In [42]:
# 컬럼 선택
df4_gu = df4_gu[['CTGG_NM', '유흥주점영업', '단란주점', '제과점영업', '집단급식소', '일반음식점', '휴게음식점']]

# 컬럼명 변경
df4_gu = df4_gu.rename(columns = {'CTGG_NM':'자치구'})

# 자치구 기준 그룹화
df4_fin = df4_gu.groupby('자치구').mean().reset_index()

# 컬럼 타입 변경
df4_fin.iloc[:, 1:] = df4_fin.iloc[:, 1:].astype(int)
df4_fin.head()

,자치구,유흥주점영업,단란주점,제과점영업,집단급식소,일반음식점,휴게음식점
0,강남구,90,118,223,231,5771,1807
1,강동구,89,60,103,181,2612,748
2,강북구,61,110,69,153,3336,626
3,강서구,68,82,157,260,3637,1121
4,관악구,125,67,109,138,3493,880


## 구별 데이터 merge

In [59]:
merge_01 = pd.merge(trash, df1, how='left', on='자치구')
merge_02 = pd.merge(merge_01, df2_fin, how='left', on='자치구')
merge_03 = pd.merge(merge_02, df4_fin, how='left', on='자치구')

df3 = pd.read_csv(PATH + '2021_구별_거주인구수_평균.csv')
merge_04 = pd.merge(merge_03, df3, how='left', on='자치구')
merge_04.head()

,자치구,PET 총량,PET 량(공공처리),PET 량(위탁),CAN 총량,CAN 량(공공처리),CAN 량(위탁),구별_총생활인구수_평균,구별_연령대0_생활인구_평균,구별_연령대10_생활인구_평균,...,구별_연령대0_거주인구_평균,구별_연령대10_거주인구_평균,구별_연령대20_거주인구_평균,구별_연령대30_거주인구_평균,구별_연령대40_거주인구_평균,구별_연령대50_거주인구_평균,구별_연령대60_거주인구_평균,구별_연령대70_거주인구_평균,구별_연령대80_거주인구_평균,구별_연령대90_거주인구_평균
0,종로구,738.6,219.3,519.3,443.5,138.9,304.6,2.451027e+08,6.419989e+06,1.145135e+07,...,424.320856,627.524064,1358.016043,1153.887701,1261.000000,1473.465241,1189.957219,699.631016,355.770053,57.299465
1,중구,712.8,565.9,146.9,484.4,373.3,111.1,1.822054e+08,3.481593e+06,5.243441e+06,...,432.181818,441.836364,1223.472727,1329.303030,1177.012121,1351.430303,1190.200000,693.096970,324.642424,55.496970
2,용산구,973.0,764.8,208.2,469.9,301.8,168.1,1.748011e+08,8.302748e+06,1.115811e+07,...,815.215909,942.193182,2082.443182,2375.125000,2163.818182,2281.630682,1814.647727,1027.750000,534.681818,90.011364
3,성동구,1671.3,1149.5,521.8,931.7,454.6,477.1,1.869606e+08,1.293545e+07,1.313938e+07,...,1159.909091,1135.812834,2600.828877,2801.604278,2586.470588,2741.893048,2195.074866,1194.069519,518.465241,67.577540
4,광진구,1315.5,0.0,1315.5,552.8,0.0,552.8,2.069475e+08,1.281745e+07,1.810854e+07,...,1229.466667,1682.448485,4146.884848,3673.745455,3414.696970,3596.430303,2903.733333,1472.460606,612.975758,81.963636


In [118]:
merge_04.to_csv('구별_쓰레기예측_모델용.csv', index = False, encoding = 'utf-8-sig')